# Python バイトコード

「Python のモジュールを import すると何が起きるか？クラス定義はどのように為されるか？」これらを深く知ろうとするとリファレンスマニュアルでは不足で、Pythonの実装(Cソースコード)を読む必要がある。実はその中間点にバイトコードが存在する。バイトコードを見ることは、リファレンスマニュアルを正しく理解する助けとなるほか、実装を調べる際の足掛かりにもなる。

Python には、disモジュールというバイトコードをダンプする標準モジュールが存在するが、これはバイトコードファイル(.pycファイル)を直接扱えない。関数をダンプすれば、その関数のバイトコードが見れるが、(既に定義されてしまっている)クラスをダンプしても、クラス内のメソッドのバイトコードは見れるが、クラスを定義する過程(ローカル辞書を構築する様やクラス変数が定義されていくさまなど)を見ることはできない

そこで、まずバイトコードファイル(.pyc)をダンプする関数を定義し、いくつかの簡単な例についてPythonのソースとバイトコードの関係から、Pythonの動作を調査する。

## バイトコードファイルのダンプ

Pythonソースファイルを pyc へコンパイルする compileall モジュールを足掛かりに調べると、py_compile というモジュールで実際に pyc ファイルを書き出していることが判明する。書き出しは以下のように行なわれる。

1. ソースファイルを文字列(codestring)として読み込む。
2. その文字列(codestring)を組み込み関数 compile でコードオブジェクト(codeobject)に変換する。
3. バイトコードファイルを新規に作成し、先頭に4バイト(0x0 4つ)書く。
4. ソースファイルのタイムスタンプを4バイトで追記。
5. コードオブジェクト(codeobject)を marshal.dump でシリアライズしたものを追記。
6. 先頭の 4 バイトをマジックコードで書き換え。

これを逆に辿ることで pyc ファイルからコードオブジェクトを取り出し、そのコードオブジェクトを dis でダンプする。

* 先の手順を見ると、1., 2. から pyc ファイルを経由することなくコードオブジェクトが得られるのでそれを dis でダンプするだけでも良いが、pycファイルしかない場合の解析という点も含めて、もとの方針で進める。

### 必要なモジュール

In [1]:
import marshal
import types
import traceback
import dis

### ダンプ処理の関数

pycファイルのダンプ処理は、pycファイルの先頭8バイト目以降のバイト列を marshal.loads でデシリアライズしてコードオブジェクトに変換し、そのコードオブジェクトに対して dis.dis を適用する。

In [2]:
def dis_pyc(pyc):
    with open(pyc, 'rb') as f:
        dis.dis(marshal.loads(f.read()[8:]))

次にソースファイルの表示・コンパイル・ダンプを実行するテスト関数を準備する。

In [3]:
import py_compile
import os.path
def tst(py):
    py = os.path.join('sample', py)
    print '<<', py, '>>'
    with open(py) as f:
        for n, s in enumerate(f):
            print '%06s\t%s' % (n+1, s[:-1])
    print
    pyc = py + 'c'
    print '<<', pyc, '>>'
    py_compile.compile(py, pyc)
    dis_pyc(pyc)

まず単純な関数が一つだけ定義されている bc01.py で試す。

In [4]:
tst('bc01.py')

<< sample\bc01.py >>
     1	def add(a, b):
     2	    return a + b

<< sample\bc01.pyc >>
  1           0 LOAD_CONST               0 (<code object add at 0000000009F2E7B0, file "sample\bc01.py", line 1>)
              3 MAKE_FUNCTION            0
              6 STORE_NAME               0 (add)
              9 LOAD_CONST               1 (None)
             12 RETURN_VALUE        


ダンプ結果から、bc01.py を import した時に、関数オブジェクトが作られ(MAKE_FUNCTION)、それをローカル辞書の 'add' に登録(STORE_NAME)していることがわかる。と同時に、add 関数の中身については LOAD_CONST で参照されているだけでダンプされていないため不十分である。LOAD_CONST の右側に表示されている `<code object add at ...>` はまさしく add 関数のコードオブジェクトであり、これにアクセスできるのであれば再帰的にダンプすることが可能である。

ここで一先ず marshal.dumps で得られる最初のコードオブジェクトの中身を見てみる。

In [5]:
def dump_code(c):
    print 'dump_code', c
    for n in dir(c):
        if n[:3] == 'co_' and n not in ('co_code', 'co_lnotab'):
            print '\tc.'+n, getattr(c, n)
    print

In [6]:
with open('sample\\bc01.pyc', 'rb') as f:
    dump_code(marshal.loads(f.read()[8:]))

dump_code <code object <module> at 0000000009F2E9B0, file "sample\bc01.py", line 1>
	c.co_argcount 0
	c.co_cellvars ()
	c.co_consts (<code object add at 0000000009F2E930, file "sample\bc01.py", line 1>, None)
	c.co_filename sample\bc01.py
	c.co_firstlineno 1
	c.co_flags 64
	c.co_freevars ()
	c.co_name <module>
	c.co_names ('add',)
	c.co_nlocals 0
	c.co_stacksize 1
	c.co_varnames ()



LOAD_CONSTのパラメータは co_consts 属性のインデックスとなっている。先のダンプ結果で、最初の LOAD_CONST 命令のパラメータは 0 であるので、`co_consts[0]` つまり `<code object add at ...>` を指していて、最後の LOAD_CONST 命令は `co_consts[1]` つまり None を指している。

co_consts の中身を再帰的にトレースすれば目的の結果が得られそうである。

### ダンプ処理の関数 (確定版)

In [7]:
def dis_code(c, upper):
    upper = upper + '.' + c.co_name
    for c2 in c.co_consts:
        if isinstance(c2, types.CodeType):
            dis_code(c2, upper)
            print
    print '[ co_name: %s ] -------' % upper, c
    print '  co_varnames :', c.co_varnames
    print '  co_names    :', c.co_names
    print '  co_freevars :', c.co_freevars
    dis.dis(c)
    
def dis_pyc(pyc):
    with open(pyc, 'rb') as f:
        dis_code(marshal.loads(f.read()[8:]), '')

再度 bc01.py をダンプしてみる。

In [8]:
tst('bc01.py')

<< sample\bc01.py >>
     1	def add(a, b):
     2	    return a + b

<< sample\bc01.pyc >>
[ co_name: .<module>.add ] ------- <code object add at 0000000009F2E430, file "sample\bc01.py", line 1>
  co_varnames : ('a', 'b')
  co_names    : ()
  co_freevars : ()
  2           0 LOAD_FAST                0 (a)
              3 LOAD_FAST                1 (b)
              6 BINARY_ADD          
              7 RETURN_VALUE        

[ co_name: .<module> ] ------- <code object <module> at 0000000009F2E930, file "sample\bc01.py", line 1>
  co_varnames : ()
  co_names    : ('add',)
  co_freevars : ()
  1           0 LOAD_CONST               0 (<code object add at 0000000009F2E430, file "sample\bc01.py", line 1>)
              3 MAKE_FUNCTION            0
              6 STORE_NAME               0 (add)
              9 LOAD_CONST               1 (None)
             12 RETURN_VALUE        


add 関数自体のコードオブジェクトがダンプされ、最後にモジュールトップレベルのコードオブジェクト(これが、import時に実行されるコード)がダンプされていることがわかる。

## 色々なダンプ結果

### クラス定義

まず、一つのクラス属性をもつクラス定義から見ていく。

In [9]:
tst('bc02.py')

<< sample\bc02.py >>
     1	class A(object):
     2	    cls_attr = 5

<< sample\bc02.pyc >>
[ co_name: .<module>.A ] ------- <code object A at 0000000009F2E930, file "sample\bc02.py", line 1>
  co_varnames : ()
  co_names    : ('__name__', '__module__', 'cls_attr')
  co_freevars : ()
  1           0 LOAD_NAME                0 (__name__)
              3 STORE_NAME               1 (__module__)

  2           6 LOAD_CONST               0 (5)
              9 STORE_NAME               2 (cls_attr)
             12 LOAD_LOCALS         
             13 RETURN_VALUE        

[ co_name: .<module> ] ------- <code object <module> at 0000000009F2E430, file "sample\bc02.py", line 1>
  co_varnames : ()
  co_names    : ('object', 'A')
  co_freevars : ()
  1           0 LOAD_CONST               0 ('A')
              3 LOAD_NAME                0 (object)
              6 BUILD_TUPLE              1
              9 LOAD_CONST               1 (<code object A at 0000000009F2E930, file "sample\bc02.py", line 1

二番目のコードオブジェクトは最初の例と同様、モジュールトップレベルのもので import 時に実行されるコードである。興味深いのは、クラス定義に関連したコードオブジェクト(最初のコードオブジェクト)が存在すること、そして、トップレベルではそのコードオブジェクトから関数を生成し(MAKE_FUNCTION)、呼び出している(CALL_FUNCTION)ところである。

最初のコードオブジェクトの役割は、クラス定義に必要なクラス属性の辞書をセットアップすることである。辞書はコードを実行する際の実行フレームの辞書(ローカル辞書)を使用し、そこに STORE_NAME 命令でクラス属性の名前と値を追加している。最後に LOAD_LOCALS と RETURN_VALUE の組み合わせで、このローカル辞書の参照を返却する。（Pythonのバイトコードはスタック型である）

モジュールトップレベルでは、(1)クラス名 'A'、(2)基底クラスのタプル(object,)、(3)最初のコードオブジェクトから得られた辞書、をもとに BUILD_CLASS 命令でクラスを生成している。これら(1),(2),(3)は丁度組込み関数 type の引数に相当するが、BUILD_CLASS 命令ではまさしく type の C実装コードの呼び出しが行われる。最後に BUILD_CLASS 命令で生成されたクラスオブジェクトを、ローカル辞書(モジュールレベルではグローバル辞書と同じ)の名前 'A' に割り当てる。

これらの動作は、Python言語リファレンスのクラス定義で次の記述に相当する:
> クラス定義は実行可能な文です。クラス定義では、まず継承リストがあればそれを評価します。継承リストの各要素の値評価結果はクラスオブジェクトか、サブクラス可能なクラス型でなければなりません。次にクラスのスイートが新たな実行フレーム内で、新たなローカル名前空間と元々のグローバル名前空間を使って実行されます (名前づけと束縛 (naming and binding) 節を参照してください ) 。 ( 通常、スイートには関数定義のみが含まれます ) クラスのスイートを実行し終えると、実行フレームは無視されますが、ローカルな名前空間は保存されます。次に、基底クラスの継承リストを使ってクラスオブジェクトが生成され、ローカルな名前空間を属性値辞書として保存します。最後に、もとのローカルな名前空間において、クラス名がこのクラスオブジェクトに束縛されます。

次にメソッドを一つ追加したコードを見る。

In [10]:
tst('bc03.py')

<< sample\bc03.py >>
     1	class A(object):
     2	    cls_attr = 5
     3	    def m(self):
     4	        pass

<< sample\bc03.pyc >>
[ co_name: .<module>.A.m ] ------- <code object m at 0000000009F2ED30, file "sample\bc03.py", line 3>
  co_varnames : ('self',)
  co_names    : ()
  co_freevars : ()
  4           0 LOAD_CONST               0 (None)
              3 RETURN_VALUE        

[ co_name: .<module>.A ] ------- <code object A at 0000000009F2E430, file "sample\bc03.py", line 1>
  co_varnames : ()
  co_names    : ('__name__', '__module__', 'cls_attr', 'm')
  co_freevars : ()
  1           0 LOAD_NAME                0 (__name__)
              3 STORE_NAME               1 (__module__)

  2           6 LOAD_CONST               0 (5)
              9 STORE_NAME               2 (cls_attr)

  3          12 LOAD_CONST               1 (<code object m at 0000000009F2ED30, file "sample\bc03.py", line 3>)
             15 MAKE_FUNCTION            0
             18 STORE_NAME               3 (

予想通り、メソッドの根底となる関数のコードオブジェクトの分だけ増えている。

### 呼び出し

次に典型的な呼び出し操作 - 関数呼び出し、インスタンス化、メソッド呼び出し、コーラブルオブジェクト呼び出しを見る。

In [11]:
tst('bc04.py')

<< sample\bc04.py >>
     1	def f(a):
     2	    return a + 1
     3	
     4	class A(object):
     5	    def __call__(self):
     6	        pass
     7	    def m(self, v):
     8	        return v * 2
     9	
    10	f(7)
    11	a = A()
    12	a.m(8)
    13	a.m(9)
    14	a()

<< sample\bc04.pyc >>
[ co_name: .<module>.f ] ------- <code object f at 0000000009F2EBB0, file "sample\bc04.py", line 1>
  co_varnames : ('a',)
  co_names    : ()
  co_freevars : ()
  2           0 LOAD_FAST                0 (a)
              3 LOAD_CONST               1 (1)
              6 BINARY_ADD          
              7 RETURN_VALUE        

[ co_name: .<module>.A.__call__ ] ------- <code object __call__ at 0000000009F2E430, file "sample\bc04.py", line 5>
  co_varnames : ('self',)
  co_names    : ()
  co_freevars : ()
  6           0 LOAD_CONST               0 (None)
              3 RETURN_VALUE        

[ co_name: .<module>.A.m ] ------- <code object m at 0000000009F2EDB0, file "sample\bc04.py", line 7>
  c

最後のコードオブジェクトのダンプを見る。命令の左側の数値ががソースコードの行番号になっている。ソースの 10行目～14行目はそれぞれ、関数呼び出し、インスタンス化、メソッド呼び出し、同一のメソッド呼び出し、オブジェクト呼び出し、となっている。これらのどれをとっても、CALL_FUNCTION が呼ばれていて、関数呼び出しと区別がないというのが特徴である。

また、メソッド呼び出しは対象となるオブジェクトを名前 a から取得し(LOAD_NAME)、名前 m の属性値を取得する(LOAD_ATTR)、という二段のステップが必要となることがわかる。また、ソースコード上で同一のメソッド呼び出し(二回目の a.m())であっても、この二段のステップは省略されない。Pythonは動的な言語であるため、メソッド呼び出し a.m() で a.m あるいは type(a).m を別の関数等に変更できるためである。

もし、同じオブジェクト a のメソッド m を何度も呼び出すのであれば、
```python
bm = a.m         # bm は bound メソッドオブジェクト
for v in ...:    # ... は引数列
    bm(v)

```
とするか、クラス A の多数の異なるインスタンスに対してメソッド m を呼び出すのであれば、
```python
um = A.m         # um は unbound メソッドオブジェクト
for o in ...:    # ... はクラスAのインスタンス列
    um(o, arg)
```
としなければならない。余談ではあるが、um (== A.m) の呼び出しでは第一引数のクラスのチェックが行なわれたあと根底の関数の呼び出しが発生するので、より効率をあげるなら、
```python
f = A.m.im_func  # f はメソッドの根底にある関数オブジェクト
for o in ...:    # ... はクラスAのインスタンス列
    f(o, arg)
```
とする方法がある。

なお、A.__dict__['m'] が関数オブジェクトの場合の a.m や A.m の属性参照(LOAD_ATTR)では、通常の属性参照の動作に加えて関数オブジェクトからメソッドオブジェクトへの変換が発生する。このメカニズムは関数オブジェクトがディスクリプタとして実装されていることによる。

### インポート文

最後に import 文について見てみる。

In [12]:
tst('bc05.py')

<< sample\bc05.py >>
     1	import pkg.subpkg.moda
     2	import pkg.subpkg.modb as _b
     3	from pkg.subpkg import modc as _c
     4	from pkg.subpkg import *

<< sample\bc05.pyc >>
[ co_name: .<module> ] ------- <code object <module> at 0000000009F2EBB0, file "sample\bc05.py", line 1>
  co_varnames : ()
  co_names    : ('pkg.subpkg.moda', 'pkg', 'pkg.subpkg.modb', 'subpkg', 'modb', '_b', 'pkg.subpkg', 'modc', '_c')
  co_freevars : ()
  1           0 LOAD_CONST               0 (-1)
              3 LOAD_CONST               1 (None)
              6 IMPORT_NAME              0 (pkg.subpkg.moda)
              9 STORE_NAME               1 (pkg)

  2          12 LOAD_CONST               0 (-1)
             15 LOAD_CONST               1 (None)
             18 IMPORT_NAME              2 (pkg.subpkg.modb)
             21 LOAD_ATTR                3 (subpkg)
             24 LOAD_ATTR                4 (modb)
             27 STORE_NAME               5 (_b)

  3          30 LOAD_CONST               

モジュールの import は IMPORT_NAME でモジュールの読み込みとモジュールトップレベルのコードオブジェクトの実行が行なわれる。多階層(pkg.subpkg.moda)のモジュールを指定した場合でも、IMPORT_NAME は最上位(pkg)のモジュールオブジェクトをスタックに積む。

2行目の import 文のように多階層の末端のモジュール(modb)に別名(\_b)を指定つける場合は、IMPORT_NAME による pkg から階層に応じて LOAD_ATTR を行なって目的のモジュール(modb)を取得する必要がある。

一方 2行目と実質的に同じ効果をもつ3行目の from 形式では、IMPORT_FROM 一発で目的のモジュール(modc)を取得している。

4行目のワイルドカード形式では、IMPORT\_STAR 命令の実装部分で、\_\_all\_\_ の検索などが行なわれる。

## 普通は dis.dis() で十分

これまでは、モジュールインポート時のクラス定義などを調べる目的もあり、バイトコードファイルを対象として、コードオブジェクトを再帰的にダンプする関数を定義した。しかし、例えば for 文の構造やリスト内包表記、キーワード付き引数を伴った関数呼び出しなどを調べるのであれば、調べたいコードを関数の内部に定義してしまえば、標準の dis.dis() で簡単に調べることができる。

In [13]:
def f():
    return [n*2 for n in range(10)]

dis.dis(f)

  2           0 BUILD_LIST               0
              3 LOAD_GLOBAL              0 (range)
              6 LOAD_CONST               1 (10)
              9 CALL_FUNCTION            1
             12 GET_ITER            
        >>   13 FOR_ITER                16 (to 32)
             16 STORE_FAST               0 (n)
             19 LOAD_FAST                0 (n)
             22 LOAD_CONST               2 (2)
             25 BINARY_MULTIPLY     
             26 LIST_APPEND              2
             29 JUMP_ABSOLUTE           13
        >>   32 RETURN_VALUE        


ちなみに、リスト内包表記をモジュールレベルで記述すると以下のようになる。

In [14]:
tst('bc06.py')

<< sample\bc06.py >>
     1	[n*2 for n in range(10)]

<< sample\bc06.pyc >>
[ co_name: .<module> ] ------- <code object <module> at 0000000009F2E430, file "sample\bc06.py", line 1>
  co_varnames : ()
  co_names    : ('range', 'n')
  co_freevars : ()
  1           0 BUILD_LIST               0
              3 LOAD_NAME                0 (range)
              6 LOAD_CONST               0 (10)
              9 CALL_FUNCTION            1
             12 GET_ITER            
        >>   13 FOR_ITER                16 (to 32)
             16 STORE_NAME               1 (n)
             19 LOAD_NAME                1 (n)
             22 LOAD_CONST               1 (2)
             25 BINARY_MULTIPLY     
             26 LIST_APPEND              2
             29 JUMP_ABSOLUTE           13
        >>   32 POP_TOP             
             33 LOAD_CONST               2 (None)
             36 RETURN_VALUE        


range や変数名 n を扱う命令が異なっていることに気づく。関数内のローカル変数の保存には、通常の辞書タイプの他に特別なストレージがあり、STORE_FAST, LOAD_FAST は後者を用いて高速に変数へのアクセスが可能となっている。

## まとめ

バイトコードファイル全体をダンプすることで、モジュールインポート時の動作を知ることができる。特にクラスがどのように定義されるかを知ることは、Python の名前とオブジェクトの関係性などを正しく理解するために有用である。

一方、バイトコードレベルでは見えない特性も多くある。Cによる実装まで踏みこんでみると、オブジェクトの呼び出し(CALL_FUNCTION系)とクラス生成(BUILD_CLASS)の共通の構造などを垣間見ることもでき、より Python の理解を深めることができるのでチャレンジしてみたい。